<a href="https://colab.research.google.com/github/AlvaroMAlves/Pred-Maintenance-Machine/blob/main/Predictive_Maintenance_of_Machines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Predictive Maintenance of Machines

## Introduction

Importar as bibliotecas necessárias:

In [1]:
import numpy
import pandas as pd

Importar a base de dados do GitHub:

In [3]:
url = 'https://raw.githubusercontent.com/AlvaroMAlves/Pred-Maintenance-Machine/main/predictive%20maintenance%20of%20machines.csv'
df = pd.read_csv(url)

Verificar o dataframe criado:

In [5]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0


The column analysis is as follows:

Machines have an ID and an ordinal categorical variable indicating the quality
of the machines, classified as Low, Medium, and High.

Data acquisition follows a time series. Observed data includes air temperature, process temperature, rotational speed, torque, and tool wear.

Starting from the "Machine Failure" column, the subsequent columns indicate binary information regarding the occurrence of a failure and specify the type of failure, following the mentioned format:

*   Tool wear failure (TWF)
*   Heat dissipation failure (HDF)
*   Power failure (PWF)
*   Overstrain failure (OSF)
*   Random failures (RNF)

